In [1]:
# Importing Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

# NLP Packages
import nltk 
from nltk.corpus import stopwords
from textblob import TextBlob 
from textblob import Word
import re
import string

# WordCloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# Sklearn Packages
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text 
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, precision_score, f1_score, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.exceptions import ConvergenceWarning

# ImbLearn Packages
from imblearn.over_sampling import SMOTE

# Pandas Settings
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows', 100)

# Solve warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [2]:
df = pd.read_csv('csv/df_no_class_imbalance.csv', index_col=0)

In [3]:
df.head()

,Hotel_Name,Negative_Review,Positive_Review,Reviewer_Score,Reviews_Clean,Score
279305,Catalonia Atenas,Nothing,Staff nice location excellent breakfast amazi...,8.8,nothing staff nice location excellent breakfa...,2
330486,Avenida Palace,No bar or restaurant,No Positive,9.2,no bar or restaurant no positive,2
223191,DoubleTree by Hilton London Islington,No Negative,The very friendly and helpful staff Made us f...,10.0,no negative the very friendly and helpful staf...,2
282475,Hotel Ronda Lesseps,Breakfast could have been better,Nice hotel super nice rooms and great locatio...,8.3,breakfast could have been better nice hotel ...,2
436370,Crowne Plaza London Docklands,Breakfast Staff not all really helpful a litt...,Location was great beautiful view close to Co...,7.9,breakfast staff not all really helpful a litt...,2


# Tokenizing

In [4]:
stop_words = stopwords.words('english')

In [5]:
df.isna().sum()

Hotel_Name         0
Negative_Review    0
Positive_Review    0
Reviewer_Score     0
Reviews_Clean      1
Score              0
dtype: int64

In [6]:
df.dropna(inplace=True)

In [7]:
# Instantiate CountVectorizer
cv = CountVectorizer(stop_words=stop_words)

# Fit and transform dataframe without data cleaning
df_cv = cv.fit_transform(df['Reviews_Clean'])
df_cv = pd.DataFrame(df_cv.toarray(), columns = cv.get_feature_names())
df_cv.index = df.index

In [8]:
df_cv.head()

,aa,aat,abadoneded,abandon,abandoned,abbey,abd,abdalla,abduction,abdulla,abdulrahim,abid,abilities,ability,abit,able,abnormal,abnoxious,abrasive,abroad,abrupt,abruptly,absence,absent,absoloutly,absolut,absolute,absoluteley,absolutely,absolutley,absorb,absoulutely,absoutely,absurd,abti,abundance,abundant,abused,abusive,abysmal,ac,acc,accent,accept,acceptable,acceptableno,accepted,accepting,accepts,acces,accesories,access,accessed,accesses,accessibility,accessible,accessing,accessories,accident,accidental,accidentally,accidently,accom,accomadated,accomendating,accomidations,accommodate,accommodated,accommodating,accommodatingno,accommodation,accommodations,accomodate,accomodated,accomodating,accomodatingno,accomodation,accomodations,accompanied,accompaniments,accor,according,accordingly,accorhotels,account,accountability,accountant,accounts,accross,accubied,accurate,accurateno,accusations,accused,accusing,accustomed,aced,acess,acessories,ache,aches,achey,achievable,achieve,aching,achive,acknowledge,acknowledged,acknowledgement,acknowledgment,acno,acocunt,acon,acoustic,acoustics,acquire,across,act,acted,acting,action,actionno,actions,activate,activated,activation,active,actively,activites,activitesno,activities,activity,acton,actual,actually,acually,acumen,ad,adagio,adam,adaptations,adapted,adapter,adapters,adapting,adaptor,adaptors,add,added,addicts,adding,addition,additional,additionally,additionl,additions,address,addressed,addressing,adds,addttional,adequate,adequately,adequatte,adevertise,adhere,adhered,aditional,adjacent,adjacently,adjoined,adjoining,adjourning,adjust,adjustable,adjusted,adjusting,adjustment,adjustments,administer,administration,adminities,admission,admit,admitted,admittedly,ado,adore,adressed,adsno,adult,adults,advance,advanced,advantage,advantages,adventure,adventurous,advert,advertise,advertised,advertisement,advertisementno,advertisided,advertising,advertize,advice,advices,advisable,advise,advised,advises,advising,advisor,aed,aeration,aerial,aerobic,aerosol,aesthetic,affair,affect,affected,affecting,affiliated,affixed,afford,affordable,afield,afraid,afre,afree,afrerad,africa,african,afrosted,afternoon,afternoons,aftershave,aftertaste,afterthought,afterwards,againno,age,aged,agedno,ageing,agency,agenda,agender,agenno,agent,agents,ages,aggrasive,aggravate,aggravated,aggressive,agianno,agile,agin,aging,agitated,ago,agoda,agony,agood,agr,agrasif,agree,agreed,agreeing,agreement,agressive,ah,aha,ahead,ahmed,ahotel,ahuge,aid,aids,aim,aimed,aiming,aimlessly,aint,air,airbnb,airco,aircon,aircond,airconditing,aircondition,airconditioned,airconditioner,airconditioning,aircondtion,aircondtioner,aired,aires,airflow,airing,airless,airline,airno,airplane,airplanes,airport,airports,airways,airy,aisle,aitport,aiva,ajala,ajax,ajoining,ajustable,ak,aka,akin,akthoudh,akward,al,ala,alarm,alarmed,alarming,alarms,alawys,albeit,albert,albertina,alberto,alcahol,alchohol,alcohol,alcoholic,alcove,aldgate,alert,alerted,alerting,ales,algate,algeria,alhtough,ali,alice,alicia,alike,alin,alittle,alkohol,alla,allan,alle,allegation,allegedly,alleregic,allergi,allergic,allergies,allergiesno,allergy,alles,alley,alll,allno,allocate,allocated,allocation,allocations,allot,allotted,allow,allowed,allowedno,allowing,allows,allready,allroundno,alltough,allways,almond,almost,almoust,aloft,aloi,alone,along,aloof,alot,aloud,already,alright,also,alsono,alter,alterations,altered,alternate,alternating,alternative,alternatively,alternatives,althiugh,altho,althogh,although,altogether,alwats,alway,always,alwful,amadeus,amaretto,amasing,amateur,amateurish,amateurs,amazaing,amazed,amazing,amazingly,amba,ambassador,ambiance,ambience,ambient,ambition,ambivalent,amble,ambulance,ambulances,amd,amenable,amend,amended,amendment,amends,amenities,amenity,american,americano,americans,amesterdam,amex,amicably,amidst,aminities,amir,amiss,ammenities,ammount,amneties,among,amongst,amount,amounting,amounts,ample,amplified,amrath,amst,amsterdam,amsterdamit,amsterdamno,amu

In [9]:
df_cv.shape

(12686, 14233)

In [10]:
# Using Sparse in the DataFrame
df_sparse = df_cv.astype('Sparse')

In [11]:
df_sparse.shape

(12686, 14233)

In [12]:
y = df.Score
X = df_sparse

In [13]:
# Running Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25)

#31s

## Evaluation Metric

In [14]:
# Evaluation function

def evaluation(y_true, y_pred):
       
# Print Accuracy, Recall, F1 Score, and Precision metrics.
    print('Evaluation Metrics:')
    print('Accuracy: ' + str(metrics.accuracy_score(y_test, y_pred)))
    print('F1 Score: ' + str(metrics.f1_score(y_test, y_pred, average="weighted")))

## Modeling With Count Vectorizer

### Logistic Regression

In [15]:
# Baseline Regression Model
lg_base = LogisticRegression()
lg_base.fit(X_train, y_train) 
y_lg_base = lg_base.predict(X_test)

# 2m 28s seconds

In [16]:
# Logistic Regression baseline evaluation
evaluation(y_test, y_lg_base)

Evaluation Metrics:
Accuracy: 0.6116015132408575
F1 Score: 0.6123075813752655


### Random Forest

In [17]:
rf_cv = RandomForestClassifier()
rf_cv.fit(X_train, y_train)

y_rf_cv = rf_cv.predict(X_test)

KeyboardInterrupt: 

In [ ]:
# Random Forest baseline evaluation
evaluation(y_test, y_rf_cv)

### Naive Bayes

In [ ]:
nb_base_cv = GaussianNB()
nb_base_cv.fit(X_train, y_train)

y_nb_base_cv = nb_base_cv.predict(X_test)

In [ ]:
# Naive Bayes baseline evaluation
evaluation(y_test, y_nb_base_cv)

## Modeling with TF-IDF

In [ ]:
def fn_tdm_tfidf(docs, xColNames = None, **kwargs):
    ''' create a term document matrix as pandas DataFrame
    with **kwargs you can pass arguments of CountVectorizer
    if xColNames is given the dataframe gets columns Names'''

    #initialize the  vectorizer
    tf = TfidfVectorizer(**kwargs)
    x1 = tf.fit_transform(docs)
    #create dataFrame
    df = pd.DataFrame(x1.toarray().transpose(), index = tf.get_feature_names())

    if xColNames is not None:
        df.columns = xColNames

    return df

In [ ]:
df_tfidf = fn_tdm_tfidf(df['Reviews_Clean']).transpose()
df_tfidf=pd.concat([df_tfidf, dummy], axis=1, sort=False)